Library

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"


import subprocess
with open(os.path.expanduser("~/successful NUMA node read from SysFS had negative value (-1)"), "r") as f:
    password = f.readline()
proc = subprocess.Popen(['sudo', '-S', 'sh', '-c', 'for a in /sys/bus/pci/devices/*; do echo 0 | sudo tee -a $a/numa_node; done'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,universal_newlines= True)
stdout, stderr = proc.communicate(input=password+'\n')
print(stdout)
print(stderr)

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

[sudo] password for user: 


Dataset

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.reshape(-1, 28 ,28, 1).astype(np.uint8), x_test.reshape(-1, 28 ,28, 1).astype(np.uint8)
validation_gen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range= 30, width_shift_range= 0.2, height_shift_range= 0.2, zoom_range= 0.2, data_format= 'channels_last', dtype= np.uint8)
validation_gen.fit(x_test)
validation_generator = validation_gen.flow(x_test, y_test, batch_size= 1, shuffle=True, seed= 2, save_to_dir= None)

save image

In [3]:
quant = True
epochs = 800
image_list = []
with open(os.path.expanduser("~/Desktop/image_data.h"), "w+") as f:
    f.write("const uint8_t image_data[" + str(epochs) + "][784] PROGMEM = {\n")
    # f.write("unsigned int image_data[784] = {\n")
    for i in range(epochs):
        x,y =validation_generator.next()
        if(epochs < 5):
            plt.imshow(x[0])
            plt.show()
        image_list.append(y)
        if(epochs > 1):
            f.write("{")
        for j in range(28):
            for k in range(28):
                if(quant == False):
                    f.write(str(x[0][j][k][0] / 255.0)) #float32
                elif(quant == True):
                    f.write(str(x[0][j][k][0]))   #uint8
                if((j < 27 and k != 27) or (j == 27 and k != 27)):
                    f.write(",")
                if(k == 27 and j == 27 and epochs > 1):
                    f.write("}")
                if(j < 27 and k == 27):
                    f.write(",")
            if(j != 27):    
                f.write("\n")
        if(i != epochs - 1):
            f.write(",")
        f.write("\n")
    f.write("};")

with open(os.path.expanduser("~/Desktop/image_list.h"), "w+") as f:
    f.write("const int image_count = " + str(epochs) + ";\n")
    f.write("const int image_list[" + str(epochs) + "] = {\n")
    for i in range(epochs):
        f.write(str(image_list[i][0]))
        if(i != epochs - 1):
            f.write(",\n")
    f.write("\n};")